In [217]:
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from importlib import reload
import InputList
import BinaryReader
import Preprocessor
import models
import Callbacks

In [235]:
reload(models)

<module 'models' from '/home/julius/dataspellprojects/oct-classifier/models.py'>

## Make this such that it can be used for prediction and training pipeline

In [229]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
reader = BinaryReader.BinaryReader()  # TODO: Normalizer
training_dataset, validation_dataset = reader.create_training_datasets(InputList.training_files)
preprocesser = Preprocessor.Preprocessor(training_dataset)

Num GPUs Available:  0


In [230]:
print("Creating Normalization Layer:")
normalization_layer = preprocesser.normalize_layer()

Creating Normalization Layer:


In [236]:
model = models.classiRaw3D(training_dataset.element_spec[0].shape, normalization_layer)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1536, 23, 28, 1)] 0         
_________________________________________________________________
permute_14 (Permute)         (None, 1, 23, 28, 1536)   0         
_________________________________________________________________
dense_14 (Dense)             (None, 1, 23, 28, 768)    1179648   
_________________________________________________________________
permute_15 (Permute)         (None, 768, 23, 28, 1)    0         
_________________________________________________________________
conv3d_28 (Conv3D)           (None, 768, 23, 28, 32)   896       
_________________________________________________________________
max_pooling3d_28 (MaxPooling (None, 384, 11, 14, 32)   0         
_________________________________________________________________
conv3d_29 (Conv3D)           (None, 384, 11, 14, 64)   5536

In [237]:
history = model.fit(
    preprocesser.batch(20).take(2),
    epochs=10,
    validation_data=Preprocessor.Preprocessor(validation_dataset).batch(20).take(2),
    # callbacks=Callbacks.my_callbacks
)
model.save('savedModels/first')

Epoch 1/10
2/2 [==============================] - 76s 41s/step - loss: 3862.4815 - sparse_categorical_crossentropy: 3862.4815 - val_loss: 21916.2012 - val_sparse_categorical_crossentropy: 21916.2012
Epoch 2/10


KeyboardInterrupt: 

In [224]:
training_dataset

<PrefetchDataset shapes: ((1536, 23, 28, 1), ()), types: (tf.uint16, tf.uint8)>

In [8]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


In [50]:
model.load_weights('checkpoints/best_model.data-00000-of-00001')

OSError: SavedModel file does not exist at: checkpoints/best_model.data-00000-of-00001/{saved_model.pbtxt|saved_model.pb}

print_function(Bag Number, dataset)

-> In einem Bag sind 7000 Instanzen

# Bag Level Model:

Ich hab ganz viele Softmaxes und von denen nehm ich einfach den Max Wert

In [58]:
test_dataset = reader.create_test_dataset([InputList.healthy_training_files[0]])
output = model.predict(test_dataset.batch(20))

KeyboardInterrupt: 

Linearisierung: